In [75]:
from create_model import create_model
import torch
import torch.nn.utils.prune as prune
import numpy as np
import torch.nn as nn
from learn_that import apply_model

from copy import deepcopy as deepcopy

In [76]:
datasize = 3

In [77]:
X = 0.1- np.random.random((datasize,10)) 

Y = np.random.random(datasize)

This is md

In [78]:
torch.manual_seed(0)
model, optimizer, loss_fn = create_model(X, optim="sparse_adam")

In [79]:
parameters = list(model.named_parameters())
t = parameters[0]


In [80]:
t

('blocks.0.linear.weight',
 Parameter containing:
 tensor([[-0.0024,  0.1696, -0.2603, -0.2327, -0.1218,  0.0848, -0.0063,  0.2507,
          -0.0281,  0.0837],
         [-0.0956, -0.0622, -0.3021, -0.2094, -0.1304,  0.0117,  0.1250,  0.1897,
          -0.2144, -0.1377],
         [ 0.1149,  0.2626, -0.0651,  0.2366, -0.0510,  0.0335,  0.2863, -0.2934,
          -0.1991, -0.0801],
         [-0.1233,  0.2732, -0.2050, -0.1456, -0.2209, -0.2962, -0.1846,  0.2718,
           0.1411,  0.1533]], requires_grad=True))

In [81]:
tx = torch.tensor(X)
tx = torch.max(tx, torch.zeros_like(tx))
# tx = tx.to_sparse()
ty = torch.tensor(Y)

In [82]:
tx, ty = tx.type(torch.FloatTensor), ty.type(torch.FloatTensor)

In [83]:
loss = loss_fn(apply_model(tx, model=model).squeeze(1), ty)

loss.backward()

In [85]:
for p in model.parameters():
    p = p.to_sparse()

In [86]:
optimizer.step()

RuntimeError: SparseAdam does not support dense gradients, please consider Adam instead

In [74]:
parameters = list(model.named_parameters())
t = parameters[0]
t

('blocks.0.linear.weight',
 Parameter containing:
 tensor([[-0.0024,  0.1696, -0.2603, -0.2327, -0.1218,  0.0848, -0.0063,  0.2507,
          -0.0281,  0.0837],
         [-0.0956, -0.0622, -0.3021, -0.2094, -0.1304,  0.0117,  0.1250,  0.1897,
          -0.2144, -0.1377],
         [ 0.1149,  0.2626, -0.0651,  0.2366, -0.0510,  0.0335,  0.2863, -0.2934,
          -0.1991, -0.0801],
         [-0.1233,  0.2732, -0.2050, -0.1456, -0.2209, -0.2962, -0.1846,  0.2718,
           0.1411,  0.1533]], requires_grad=True))

In [69]:
t[1].grad.to_sparse()

tensor(indices=tensor([[0, 0, 0, 2, 2, 2, 2],
                       [1, 4, 6, 1, 4, 6, 8]]),
       values=tensor([-0.0012, -0.0019, -0.0020,  0.0012,  0.0019,  0.0020,
                       0.0021]),
       size=(4, 10), nnz=7, layout=torch.sparse_coo)

In [ ]:
t

In [ ]:
test =  nn.Conv2d(5, 3, 2)

In [ ]:
list(test.named_parameters())[0][1].grad

In [ ]:
grad = t[1]
y = torch.zeros_like(x)
torch.where(grad != 0, x, y)

In [ ]:
t[1]